# Bioinformatics Algorithm

## 过程化考核4: Hidden Markov Model

> Last Modified Time 2019-11-04 15:18 By ZeFengZhu

### 问题描述

Make a report about HMM and the relevant knowledge in the class.

### 基础概念

#### Finite State Machines

> Finite State Machines. Brilliant.org. Retrieved 15:50, November 4, 2019, from https://brilliant.org/wiki/finite-state-machines/

#### Random Walks

> Random Walks . Brilliant.org. Retrieved 15:52, November 4, 2019, from https://brilliant.org/wiki/the-random-walk/

#### HMM

> https://ww2.mathworks.cn/help/stats/hidden-markov-models-hmm.html

> https://www.cnblogs.com/Determined22/p/6750327.html

> http://www.randomservices.org/random/markov/General.html

> Markov Chains. Brilliant.org. Retrieved 15:45, November 4, 2019, from https://brilliant.org/wiki/markov-chains/

> A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition, _LAWRENCE R. RABINER, FELLOW, IEEE_ 

> A Revealing Introduction to Hidden Markov Models, _Mark Stamp∗ Department of Computer Science San Jose State University_

> An Introduction to Conditional Random Fields for Relational Learnin, _Charles Sutton, Andrew McCallum Department of Computer Science University of Massachusetts, USA_

* Stochastic Processes

### HMM 概述

![fig of HMM](../../docs/figs/hmm.png)

> 图中阴影部分为观测变量，白圈代表隐变量

1. Dynamic Model (time + mixture), 模型用如下$\lambda$表达
2. $\lambda = (\pi, A, B)$
    * $\pi \rightarrow$ 初始probability distribution
    * $A \rightarrow$ 状态转移矩阵
    * $B \rightarrow$ 发射矩阵
3. 观测序列 $O \, \{o_1, o_2, ..., o_t, o_{t+1},...\}$
    * 观测变量 $o$
    * 值域(观测值集合) $V = \{v_{1}, v_{2}, ..., v_{M}\}$
4. 状态序列 $i \,\,\, \{i_1, i_2, ..., i_t, i_{t+1},...\}$
    * 状态变量 $v$ 
    * 值域(状态值集合) $Q = \{q_{1}, q_{2}, ..., q_{N}\}$
5. $A = [a_{ij}] ,\,\, a_{ij}=P(i_{t+1}=q_j|i_t=q_i)$
6. $B = [b_{j(k)}], \,\, b_{j(k)} = P(o_t=v_k|i_t=q_j)$

#### 两个假设
1. 齐次马尔可夫假设 (无后效性)
$$P(i_{t+1}|i_t,i_{t-1},...i_1,\,\,o_t,o_{t_1},...,o_1)=P(i_{t+1}|i_t)$$
    * 对t+1时刻，该时刻状态仅与t时刻的状态有关，与前面其他任意时刻的状态以及观测值无关系

2. 观测独立假设 
$$P(o_t|i_t,i_{t-1},...i_1,\,\,o_t,o_{t_1},...,o_1)=P(o_t|i_t)$$
    * 对于t时刻，观测变量仅与t时刻的状态有关

#### 三个问题
1. Evalution
    * 已知$\lambda$,求该观测序列$O=o_1o_2...o_t$出现的概率
    * $P(O|\lambda)$
    * Forward/Backward Algorithm
2. Learning
    * 参数估计问题：$\lambda$如何求
    * $\lambda = arg\,maxP(O|\lambda)$
    * EM算法
3. Decoding
    * 已知观测序列$O$，找到一状态序列$I=i_1i_2...i_t$使得$P(I|O)$最大
    * $I=arg\,maxP(I|O)$
    * 预测: $P(i_{t+1}|o_1o_2...o_t)$
    * 滤波: $P(i_t|o_1o_2...o_t)$
    

### HMM Evaluation Problem

> Given $\lambda$，求$P(O|\lambda)$

$P(O|\lambda)=\sum_{I}P(I,O|\lambda)=\sum_{I}P(O|I,\lambda)\,P(I|\lambda)$

#### 推理

1. $P(i_t|i_1,i_2,...,i_{t-1}\,,\lambda)=P(i_t|i_{t-1})=a_{i_{t-1},\,i_t}$
2. 进行联合分布递归展开

因为:

$$P(I|\lambda)=P(i_1,i_2,...,i_t|\lambda)=P(i_t|i_1,i_2,...,i_{t-1}\,,\lambda)\,P(i_1,i_2,...,i_{t-1}\,,\lambda)=\pi(a_{i_t})\,\prod_{t=2}^{T}a_{i_{t-1},\,i_t}$$

$$P(O|I,\lambda)=\prod_{t=1}^{T}b_{i_t}(o_t)$$

所以:

$$P(O|\lambda)=\sum_{I}\pi(a_{i_1})\prod_{t=2}^{T}a_{i_{t-1},\,i_t}\prod_{t=1}^{T}b_{i_t}(O_t)$$

该公式反映了这个求解过程是指数级算法($O(N^T)$)，需要优化

#### Forward Algorithm

> 结合两个假设推导

1. 引入记号$\alpha_{t}(i)$，记为初始到t时刻的观测序列以及t时刻的状态的联合概率
$$\alpha_{t}(i)=P(o_1,...o_t,\,i_t=q_i|\lambda) \Rightarrow \alpha_{T}(i)=P(O,\,i_t=q_i|\lambda)$$
2. 递推式: $\alpha_{t+1}(j)=\sum_{i=1}^{N}b_{j}(O_{t+1})\,a_{ij}\,\alpha_{t}(i)$
3. $P(O|\lambda)=\sum_{i=1}^{N} P(O,i_t=q_i|\lambda)=\sum_{i=1}^{N}\alpha_{T}(i)$


#### Backward Algorithm

> 结合两个假设推导, 为前向算法的变体, 从最后一个观察到的符号开始，在网络中后向移动，直到到达位置T

1. 引入记号$\beta_{t}(i)$，记为t时刻到最终T时刻的观测序列以及t时刻的状态的联合概率
$$\beta_{t}(i)=P(o_{t+1},...o_T,\,i_t=q_i|\lambda)$$
2. 递推式: $\beta_{t}(i)=\sum_{j=1}^{N}b_{j}(o_{t+1})\,a_{ij}\,\beta_{t+1}(j)$
3. $P(O|\lambda)=\sum_{i=1}^{N} P(o_1,...,o_T,i_t=q_i|\lambda)=\sum_{i=1}^{N}b_{i}(o_1)\pi_{i}\beta_{1}(i)$